In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import numpy as np
import torch as th
from syft import VirtualMachine
from syft.core.plan.plan_builder import PLAN_BUILDER_VM, make_plan, build_plan_inputs, ROOT_CLIENT
from syft import logger
from syft import Plan
from syft.core.node.common.service.set_result_permission_service import SetResultPermissionMessage
from syft import serialize, deserialize
import syft
logger.remove()

# Result permission example

In [3]:
node = VirtualMachine(name="alice")

We define two client, data scientist (ds) and data owner (owner)

In [4]:
owner = node.get_root_client()
ds = node.get_client()

In [5]:
@make_plan
def train(x = th.rand([64*3, 1, 28, 28])):
    return x+1

We create a dataset

In [6]:
x = th.rand([64*3, 1, 28, 28])
x_ptr = x.send(owner)

And let the DS search for a pointer to the data (it will not have read access)

In [7]:
x_ds = ds.store[0]

The ds in turn create a plan, which he wants to apply on the dataset

In [8]:
train_ptr = train.send(ds)

We can execute the plan, but we cannot download the result

In [9]:
try:
    res_ptr = train_ptr(x=x_ptr)
    res_ptr.get()
except:
    print("No permissions")
else:
    raise ValueError("You should not have permissions to download")

No permissions


Now, the DO finds a pointer to plan, and allows execution on x

In [10]:
train_ptr_do = [y for y in owner.store if y.__name__ == "PlanPointer"][0]
train_ptr_do.set_result_permissions("__call__", {"x": x_ptr}, ds.verify_key)

'success'

And we can verify that the result is the same

In [11]:
res_ptr = train_ptr(x=x_ptr)
result, = res_ptr.get()
assert th.equal(result, x+1)

# serialization

In [17]:
msg = SetResultPermissionMessage(syft.core.common.UID(), {"x": plan_do.id_at_location}, "__call__", ds.verify_key,
                                 owner.address, owner.address)

In [18]:
pb = serialize(msg)

In [17]:
reconstructed_message = deserialize(pb)

In [18]:
r = reconstructed_message

assert r.inputs == msg.inputs
assert r.id_at_location == msg.id_at_location
assert r.method_name == msg.method_name
assert r.verify_key == msg.verify_key
assert r.address == msg.address
assert r.reply_to == msg.reply_to